In [81]:
import requests
from lxml import etree
import json

In [84]:
class BOESDKParser():

    def __init__(self, protocol='http', host='localhost', port='8080', content_type='application/xml'):
        base_url = protocol + '://' + host + ':' + port
        self.bip_url = base_url + '/biprws'
        self.webi_url = self.bip_url + '/raylight/v1'
        self.sl_url = self.bip_url + '/sl/v1'
        self.headers = {
            'accept' : content_type
            }
      
    def _get_auth_info(self):
        return requests.get(self.bip_url + '/logon/long',
                            headers=self.headers)

    def _send_auth_info(self, username, password):
        '''Helper function to retrieve a log in token'''
        auth_template = self._get_auth_info().text
        #print(auth_template)
        root = etree.fromstring(auth_template)
        root[3].text = username
        root[0].text = password

        print(etree.tostring(root))
        print(self.bip_url + '/logon/long')
        return requests.post(self.bip_url + '/logon/long',headers=self.headers, data=etree.tostring(root))       
    
    def set_logon_token(self, username, password):
        resp = self._send_auth_info(username, password)
        print(str(resp.text))
        
        if resp.status_code == 200:
            root = etree.fromstring(resp.text)
            
            # Set logon token in headers
            # self.headers['X-SAP-LogonToken'] = root[0].text
        else:
            
            # Crude exception handling
            raise Exception("Could not log on and set the logon token!")
            
    def get_universes(self):
        resp = requests.get(self.webi_url + '/universes', headers=self.headers)
        
        if resp.status_code == 200:
            root = etree.fromstring(resp.text)
            # Iterate over the children elements and convert them into a dict
            # of dicts
            univs = dict()
            for index, univ in enumerate(root):
                univs[index] = dict()
                for child in univ:
                    univs[index][child.tag] = child.text

            return univs
        else:
            # Crude Exception handling
            raise Exception(('Could not retrieve universes - have you set a '
                             'valid logon token?'))

In [85]:
parser = BOESDKParser("http","172.29.67.6","6405", "application/xml")
parser.set_logon_token("Administrator", "Admin@123")

b'<attrs xmlns="http://www.sap.com/rws/bip"><attr name="password" type="string">Admin@123</attr><attr name="clientType" type="string"/><attr name="auth" type="string" possibilities="secEnterprise,secLDAP,secWinAD,secSAPR3">secEnterprise</attr><attr name="userName" type="string">Administrator</attr></attrs>'
http://172.29.67.6:6405/biprws/logon/long



Exception: Could not log on and set the logon token!

<attrs xmlns="http://www.sap.com/rws/bip"><attr name="password" type="string"></attr><attr name="clientType" type="string"></attr><attr name="auth" type="string" possibilities="secEnterprise,secLDAP,secWinAD,secSAPR3">secEnterprise</attr><attr name="userName" type="string"></attr></attrs>
b'<attrs xmlns="http://www.sap.com/rws/bip"><attr name="password" type="string">Admin@123</attr><attr name="clientType" type="string"/><attr name="auth" type="string" possibilities="secEnterprise,secLDAP,secWinAD,secSAPR3">secEnterprise</attr><attr name="userName" type="string">Administrator</attr></attrs>'
http://172.29.67.6:6405/biprws/logon/long



Exception: Could not log on and set the logon token!

In [ ]:
univ = parser.get_universe()

In [ ]:
print(univ)